<a href="https://colab.research.google.com/gist/analyticsindiamagazine/8a9015b7f8d12d5b414e89007d9b069a/starter_notebook_for_participants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Required Files

In [ ]:
#imports needed
import pandas as pd
import numpy as np
import os
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext

# our hyperparameters (add more when/if you need them)
# device = torch.device('cuda:0') # Use the MLT GPU when you can!
device = torch.device('cpu')

batch_size = 8
learning_rate = 0.001
epochs = 3

In [ ]:
## change it to the unzip path of the downloaded dataset..
data_folder = r'/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/Scripts'

In [ ]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

# Read Train Files

In [ ]:
train_df = pd.read_csv('Train.csv')

In [ ]:
train_df.Labels.value_counts()

In [ ]:
test_df = pd.read_csv('/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/Test.csv')
test_df

In [ ]:
train_df.Labels.value_counts()

In [ ]:
## let's read the text scripts in the train and test dataframes..
import string

train_df['text'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['text'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
def clean_text(text):
    tokenized_text = []
    sw = stopwords.words(('english'))
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces and convert to lowercase 
    text = ' '.join(text.split()).lower() 
    
    no_stopword_text = [word for word in text.split() if not word in sw]
     
    
#    for tok in text:
#        tokens = word_tokenize(tok)
#        for each_token in tokens:
#            if each_token in sw:
#                tokens.remove(each_token)
#            else:
#                continue
#        tokenized_text.append(tokens)
    
       
    return ' '.join(no_stopword_text)

In [ ]:
#train_df.head(7)
#train_df['clean_text'] = train_df['text'].apply(lambda x: clean_text_text(x))
#test_df['text']= test_df.text.apply(clean_text)
train_df['clean_text'] = train_df['text'].apply(lambda x: clean_text(x))

In [ ]:
test_df['clean_text'] = test_df['text'].apply(lambda x: clean_text(x))

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv(r'/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/data/proc_training.csv', index = False, header=True,sep='\t')
test_df.to_csv(r'/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/data/proc_testing.csv', index = False, header=True,sep='\t')

In [ ]:
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
import spacy

def dataloader(path):

    ddir = path # '/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/data/'
    
    FILE = Field(batch_first = True,
                 lower = True)
    
    
    LABEL = Field(batch_first = True,
                  sequential = False,
                   use_vocab = False)
    
    TEXT = Field(tokenize = 'spacy',
                 tokenizer_language= 'en',
                 batch_first = True,
                 lower = True,
                 use_vocab = True)    
    
    

    fields = [('file',FILE),('genre',LABEL),('script', TEXT)]

    train, test = TabularDataset.splits(path=path,
                                        train = 'proc_training.csv',
                                        test = 'proc_testing.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True,
                                        csv_reader_params = {'delimiter': '\t'})
                                                             
    
    FILE.build_vocab(train, test)
    
    LABEL.build_vocab(train, test)
    TEXT.build_vocab(train, test)
    
    
    train_iter, test_iter = BucketIterator.splits((train, test),
                                                   batch_size = 10,
                                                   sort_within_batch = True,
                                                   sort_key = lambda x: len(x.script),
                                                   shuffle = True,
                                                   device = device)
    
    return train_iter, test_iter, FILE, LABEL, TEXT
    


In [ ]:
train_iter, test_iter, FILE, LABEL, TEXT = dataloader('/home/gusmavko@GU.GU.SE/MovieScriptsParticipantsData/data/')

In [ ]:
num_file = 'len of file is =', len(FILE.vocab)
num_label = len(LABEL.vocab)
num_script = len(TEXT.vocab)
num_file
num_script

num_file,num_label, num_script

batch = next(iter(train_iter))
LABEL.vocab.stoi


In [ ]:
batch[0]

In [1]:
!ls -lh

total 535M
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE 131M Oct 19 09:38 HATE.ipynb
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE 247K Mar 30  2020 Movie_Scripts_Sample_Submission.xlsx
drwxr-xr-x. 2 gusmavko@GU.GU.SE domain users@GU.GU.SE 216K Mar 30  2020 Scripts
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE  16K Oct 19 09:53 StarterNotebookV2.ipynb
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE  13K Mar 30  2020 Test.csv
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE  31K Mar 30  2020 Train.csv
drwxr-xr-x. 2 gusmavko@GU.GU.SE domain users@GU.GU.SE 4.0K Sep 23 10:07 data
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE 236M Sep 23 08:42 proc_testing.csv
-rw-r--r--. 1 gusmavko@GU.GU.SE domain users@GU.GU.SE 166M Sep 23 08:59 proc_training.csv


# Import the Modeling Libraries 

#### Follow the sample submission format for your submission file

#### Please verify the following before submitting your solution to avoid an invalid sibmission.

1. The format of the file is excel(.xlsx)

2. The file doesn’t contain additional styling elements such as bold headings or table borders

3. The length of the submission exactly matches with that of the sample submission and test set

4. The file name doesnot have any spaces or special characters

5. All the columns are present